# Flash Evaluation on Unicorn Dataset:

This notebook is dedicated to evaluating Flash on the Unicorn datasets, which are graph-level in nature. We employ Flash in graph-level detection mode to analyze this dataset effectively. Upon completion of the notebook execution, the results will be presented.

## Dataset Access: 
- The Unicorn dataset can be accessed at the following link: [Unicorn Dataset](https://github.com/margoseltzer/shellshock-apt).
- Please download the required data files from the provided link.

## Data Parsing and Execution:
- Utilize the parser included in this notebook to process the downloaded files.
- To obtain the evaluation results, execute all cells within this notebook.

## Model Training and Execution Flexibility:
- By default, the notebook operates using pre-trained model weights.
- Additionally, this notebook offers the flexibility to set parameters for training Graph Neural Networks (GNNs) and word2vec models from scratch.
- You can then utilize these freshly trained models to conduct the evaluation. 

Follow these guidelines for a thorough and efficient analysis of the Unicorn datasets using Flash.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
%matplotlib inline

cuda


In [2]:
Train_Gnn = True
Train_Word2vec = False
Download_data = False
Parse_data = False

In [3]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os

In [4]:
import os.path as osp
import csv
def show(str):
	print (str + ' ' + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))

def parse_data():
    # Unzip all the tar files
    for i in range(3):
        os.system('tar -zxvf camflow-attack-' + str(i) + '.gz.tar')
    for i in range(13):
        os.system('tar -zxvf camflow-benign-' + str(i) + '.gz.tar')

    os.system('rm error.log')
    os.system('rm parse-error-camflow-*')
    
    show('Start processing.')
    for i in range(25):
        show('Attack graph ' + str(i+125))
        f = open('camflow-attack.txt.'+str(i), 'r')
        fw = open('unicorn/'+str(i+125)+'.txt', 'w')
        for line in f:
                tempp = line.strip('\n').split('\t')
                temp = []
                temp.append(tempp[0])
                temp.append(tempp[2].split(':')[0])
                temp.append(tempp[1])
                temp.append(tempp[2].split(':')[1])
                temp.append(tempp[2].split(':')[2])
                temp.append(tempp[2].split(':')[3])
                fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
        f.close()
        fw.close()
        os.system('rm camflow-attack.txt.' + str(i))

    # 125 
    for i in range(125):
        show('Benign graph ' + str(i))
        f = open('camflow-normal.txt.'+str(i), 'r')
        fw = open('unicorn/'+str(i)+'.txt', 'w')
        for line in f:
                tempp = line.strip('\n').split('\t')
                temp = []
                temp.append(tempp[0])
                temp.append(tempp[2].split(':')[0])
                temp.append(tempp[1])
                temp.append(tempp[2].split(':')[1])
                temp.append(tempp[2].split(':')[2])
                temp.append(tempp[2].split(':')[3])
                fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
        f.close()
        fw.close()
        os.system('rm camflow-normal.txt.' + str(i))
    show('Done.')

if Parse_data:
    parse_data()

In [5]:
from tqdm import tqdm 
def prepare_graph(df):
    def process_node(node, action, node_dict, label_dict, dummies, node_type):
        node_dict.setdefault(node, []).append(action)
        label_dict[node] = dummies.get(getattr(row, node_type), -1)  

    nodes = {}
    labels = {}
    edges = []
    dummies = {
        "7998762093665332071": 0,
        "14709879154498484854": 1,
        "10991425273196493354": 2,
        "14871526952859113360": 3,
        "8771628573506871447": 4,
        "7877121489144997480": 5,
        "17841021884467483934": 6,
        "7895447931126725167": 7,
        "15125250455093594050": 8,
        "8664433583651064836": 9,
        "14377490526132269506": 10,
        "15554536683409451879": 11,
        "8204541918505434145": 12,
        "14356114695140920775": 13
    }

    for row in df.itertuples():
        process_node(row.actorID, row.action, nodes, labels, dummies, 'actor_type')
        process_node(row.objectID, row.action, nodes, labels, dummies, 'object')

        edges.append((row.actorID, row.objectID))

    features = [nodes[node] for node in tqdm(nodes)]
    feat_labels = [labels[node] for node in tqdm(nodes)]
    edge_index = [[], []]
    node_index_map = {node: i for i, node in enumerate(nodes.keys())}
    for src, dst in tqdm(edges):
        src_index = node_index_map[src]
        dst_index = node_index_map[dst]
        edge_index[0].append(src_index)
        edge_index[1].append(dst_index)

    return features, feat_labels, edge_index, list(nodes.keys())


In [6]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn.functional as F
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, 20, normalize=True)
        self.linear = nn.Linear(in_features=20,out_features=out_channel)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.linear(x)
        return F.softmax(x, dim=1)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/unicorn/unicorn.model')
        self.epoch += 1

In [8]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [9]:
logger = EpochLogger()
saver = EpochSaver()

In [10]:
if Train_Word2vec:
    comb_data = []
    for i in range(20):
        f = open(f"unicorn/{i}.txt")
        data = f.read().split('\n')
        data = [line.split('\t') for line in data]
        comb_data = comb_data + data

    df = pd.DataFrame (comb_data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    phrases,labels,edges,mapp = prepare_graph(df)
    
    word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=8,epochs=300,callbacks=[saver,logger])

In [11]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,14).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
from collections import Counter
import math

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        position = torch.arange(max_len, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, device=device) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model, device=device)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]

def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(20)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float, device=device)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/unicorn/unicorn.model")

In [15]:
from torch_geometric import utils

################################## Training Main Model #####################################
if Train_Gnn:
    for i in range(1):
        f = open(f"content/unicorn/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df.sort_values(by='timestamp', ascending=True,inplace=True)
        df = df.dropna()
        phrases,labels,edges,mapp = prepare_graph(df)
        print(labels)
        print(labels[0])
        criterion = CrossEntropyLoss()

        nodes = [infer(x) for x in phrases]
        nodes = np.array(nodes)  

        graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
        graph.n_id = torch.arange(graph.num_nodes).to(device)
        mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

        for m_n in range(20):
            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            total_loss = 0
            for subg in loader:
                model.train()
                optimizer.zero_grad() 
                out = model(subg.x, subg.edge_index) 
                print(out)
                print(out[0])
                print(subg.y[0])
                loss = criterion(out, subg.y) 
                loss.backward() 
                optimizer.step()      
                total_loss += loss.item() * subg.batch_size
                break
            break
            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            for subg in loader:
                model.eval()
                out = model(subg.x, subg.edge_index)
                sorted, indices = out.sort(dim=1,descending=True)
                conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
                conf = (conf - conf.min()) / conf.max()
                pred = indices[:,0]
                cond = (pred == subg.y).to(device)
                mask[subg.n_id[cond]] = False

            print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified out of {len(nodes)}\n')
            torch.save(model.state_dict(), f'trained_weights/unicorn/unicorn{m_n}.pth')

100%|██████████| 116685/116685 [00:00<00:00, 1625155.61it/s]


[5, 1, 0, 1, 5, 8, 2, 2, 1, 0, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 2, 2, 5, 5, 0, 5, 0, 1, 1, 0, 5, 2, 2, 1, 1, 0, 5, 5, 0, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 1, 0, 4, 0, 4, 1, 2, 0, 1, 6, 0, 1, 5, 1, 2, 5, 5, 0, 11, 1, 0, 2, 2, 1, 1, 0, 5, 5, 0, 0, 4, 1, 0, 1, 2, 0, 6, 6, 1, 2, 5, 5, 0, 1, 0, 0, 2, 2, 1, 0, 1, 0, 5, 5, 0, 4, 5, 1, 5, 1, 2, 0, 0, 4, 0, 4, 6, 2, 1, 5, 0, 4, 1, 1, 2, 0, 5, 0, 1, 5, 5, 0, 1, 0, 2, 3, 1, 1, 0, 5, 5, 5, 1, 5, 5, 5, 1, 1, 2, 2, 5, 2, 0, 2, 5, 1, 0, 2, 2, 2, 2, 5, 5, 2, 9, 5, 1, 1, 5, 5, 5, 2, 5, 5, 0, 0, 5, 5, 1, 2, 2, 0, 5, 0, 5, 5, 1, 1, 1, 0, 1, 5, 1, 2, 0, 2, 5, 1, 2, 5, 2, 2, 2, 4, 5, 9, 1, 2, 1, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 5, 2, 9, 1, 2, 1, 1, 0, 2, 1, 0, 5, 5, 2, 1, 2, 5, 1, 2, 2, 1, 2, 9, 1, 2, 1, 1, 10, 5, 0, 5, 1, 5, 1, 0, 0, 2, 5, 5, 5, 5, 1, 2, 2, 2, 2, 9, 6, 1, 2, 1, 1, 0, 1, 1, 5, 5, 1, 0, 2, 5, 5, 5, 1, 2, 5, 2, 2

### Validation

In [14]:
from torch.utils.data import Dataset, DataLoader
class PhraseDataset(Dataset):
    def __init__(self, phrases):
        self.phrases = phrases

    def __len__(self):
        return len(self.phrases)

    def __getitem__(self, idx):
        return self.phrases[idx]

def batch_infer(phrases):
    return [infer(phrase) for phrase in phrases] 

In [ ]:
from tqdm import tqdm

for i in range(95,98):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    print("Preparing Graph")
    phrases,labels,edges,mapp = prepare_graph(df)
    print("Done")
    print("Inferring words")
    # dataset = PhraseDataset(phrases)
    # loader = DataLoader(dataset, batch_size=10, num_workers=4)
    # nodes = []
    # for batch in tqdm(loader):
    #     batch_embeddings = batch_infer(batch)
    #     nodes.extend(batch_embeddings)
    nodes = [infer(x) for x in tqdm(phrases)]
    nodes = np.array(nodes)  
    print("Done!")

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)
    print("Evaluating...")
    for m_n in tqdm(range(20)):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False] * len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)
    print("Done!")
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 95
Preparing Graph


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900266/900266 [00:00<00:00, 2905700.13it/s]


Done
Inferring words


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230438/230438 [00:28<00:00, 8109.90it/s]


Done!
Evaluating...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 13.22it/s]


Done!
14066 6.104027981496108
Graph #: 96


### Testing

In [22]:
thresh = 330

In [23]:
correct_benign = 0

for i in range(100,125):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in tqdm(phrases)]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if flag.sum().item() <= thresh:
        correct_benign = correct_benign + 1
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)
    

Graph #: 100


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238847/238847 [00:27<00:00, 8717.15it/s]


15362 6.431732447968784
Graph #: 101


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 241345/241345 [00:27<00:00, 8871.73it/s]


15388 6.375934865027243
Graph #: 102


...

13779 6.219419719428747
Graph #: 123


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 206983/206983 [00:23<00:00, 8849.52it/s]


13281 6.4164689853756105
Graph #: 124


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 243711/243711 [00:28<00:00, 8632.75it/s]


15411 6.3234732941886085


In [24]:
correct_attack = 0

for i in range(125,150):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if  flag.sum().item() > thresh:
        correct_attack = correct_attack + 1
   
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 125


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 973950/973950 [00:00<00:00, 2458638.36it/s]


15348 6.11708063641871
Graph #: 126


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 927125/927125 [00:00<00:00, 2527246.65it/s]


14410 6.162175106694149
Graph #: 127


...

14610 6.362242853907923
Graph #: 148


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 960381/960381 [00:00<00:00, 2502954.53it/s]


14947 6.036070234383834
Graph #: 149


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 893784/893784 [00:00<00:00, 2410732.16it/s]


14502 6.4473391721869024


In [25]:
TP = correct_attack
FP = 25 - correct_benign
TN = correct_benign
FN = 25 - correct_attack

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Number of True Positives (TP): {TP}")
print(f"Number of False Positives (FP): {FP}")
print(f"Number of False Negatives (FN): {FN}")
print(f"Number of True Negatives (TN): {TN}\n")

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TPR  
print(f"Precision: {precision}")
print(f"Recall: {recall}")

fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"Fscore: {fscore}\n")

Number of True Positives (TP): 25
Number of False Positives (FP): 25
Number of False Negatives (FN): 0
Number of True Negatives (TN): 0

Precision: 0.5
Recall: 1.0
Fscore: 0.6666666666666666

